# Imports

In [1]:
import pandas as pd
import glob
import os
import numpy as np
from featureranker.utils import *
from featureranker.plots import *
from featureranker.rankers import *
from tqdm.auto import tqdm
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from glob import glob
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from torch.utils.data import Dataset, DataLoader
import pybaseball as pyb

import warnings
warnings.filterwarnings('ignore')

C:\Users\vile3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Functions

In [2]:
import pybaseball as pyb
import pandas as pd


import pybaseball as pyb
import pandas as pd

import pybaseball as pyb
import pandas as pd
import time  # Import the time module

# Initialize a list to store all data
all_data = []

# Loop over the years from the beginning year to the current year
for year in range(2008, 2015):  # Replace 2024 with the current year
    try:
        data = pyb.batting_stats_bref(year)
        print(f"Data fetched for year: {year}")
        all_data.append(data)
    except Exception as e:
        print(f"An error occurred for year: {year}. Error: {str(e)}")
    
    time.sleep(4)  # Pause execution for 1 second

# Concatenate all dataframes in the list
all_data_df_1 = pd.concat(all_data)

print(all_data_df_1)




Data fetched for year: 2008
Data fetched for year: 2009
Data fetched for year: 2010
Data fetched for year: 2011
Data fetched for year: 2012
Data fetched for year: 2013
Data fetched for year: 2014
                   Name  Age  #days     Lev          Tm    G   PA   AB    R  \
1         David Aardsma   26   5659  Maj-AL      Boston    1    1    1    0   
2    Reggie Abercrombie   27   5567  Maj-NL     Houston   32   60   55   10   
3           Bobby Abreu   34   5567  Maj-AL    New York  156  684  609  100   
4          Manny Acosta   27   5651  Maj-NL     Atlanta    4    5    5    0   
5            Mike Adams   29   5596  Maj-NL   San Diego    2    2    2    0   
..                  ...  ...    ...     ...         ...  ...  ...  ...  ...   
986        Brad Ziegler   34   3510  Maj-NL     Arizona    1    1    1    0   
987      Ryan Zimmerman   29   3376  Maj-NL  Washington   61  240  214   26   
988   Jordan Zimmermann   28   3376  Maj-NL  Washington   29   67   55    3   
989         Be

In [3]:
import pybaseball as pyb
import pandas as pd
import time  # Import the time module

all_data=[]
# Loop over the years from the beginning year to the current year
for year in range(2015, 2018):  # Replace 2024 with the current year
    try:
        data = pyb.batting_stats_bref(year)
        print(f"Data fetched for year: {year}")
        all_data.append(data)
    except Exception as e:
        print(f"An error occurred for year: {year}. Error: {str(e)}")
    
    time.sleep(4)  # Pause execution for 1 second

# Concatenate all dataframes in the list
all_data_df_2 = pd.concat(all_data)

print(all_data_df_2)

Data fetched for year: 2015
Data fetched for year: 2016
Data fetched for year: 2017
                  Name  Age  #days     Lev                Tm    G   PA   AB  \
1        David Aardsma   33   3097  Maj-NL           Atlanta    1    1    1   
2    Jos\xc3\xa9 Abreu   28   3006  Maj-AL           Chicago  154  668  613   
3        Dustin Ackley   27   3005  Maj-AL  New York,Seattle  100  264  238   
4     Cristhian Adames   23   3005  Maj-NL          Colorado   24   58   53   
5         Austin Adams   28   3079  Maj-AL         Cleveland    1    1    1   
..                 ...  ...    ...     ...               ...  ...  ...  ...   
992     Bradley Zimmer   24   2298  Maj-AL         Cleveland   96  332  299   
993     Ryan Zimmerman   32   2277  Maj-NL        Washington  142  576  524   
994  Jordan Zimmermann   31   2311  Maj-AL           Detroit    3    6    6   
995        Ben Zobrist   36   2277  Maj-NL           Chicago  126  496  435   
996        Mike Zunino   26   2279  Maj-AL     

In [4]:
import pybaseball as pyb
import pandas as pd
import time  # Import the time module

all_data=[]
# Loop over the years from the beginning year to the current year
for year in range(2018, 2021):  # Replace 2024 with the current year
    try:
        data = pyb.batting_stats_bref(year)
        print(f"Data fetched for year: {year}")
        all_data.append(data)
    except Exception as e:
        print(f"An error occurred for year: {year}. Error: {str(e)}")
    
    time.sleep(4)  # Pause execution for 1 second

# Concatenate all dataframes in the list
all_data_df_3 = pd.concat(all_data)

print(all_data_df_3)

Data fetched for year: 2018
Data fetched for year: 2019
Data fetched for year: 2020
                  Name  Age  #days     Lev                    Tm    G   PA  \
1    Jos\xc3\xa9 Abreu   31   1927  Maj-AL               Chicago  128  553   
2     Ronald Acuna Jr.   20   1913  Maj-NL               Atlanta  111  487   
3         Willy Adames   22   1913  Maj-AL             Tampa Bay   85  323   
4           Lane Adams   28   1913  Maj-NL               Atlanta   26   29   
5           Matt Adams   29   1915  Maj-NL  St. Louis,Washington  120  337   
..                 ...  ...    ...     ...                   ...  ...  ...   
607   Mike Yastrzemski   29   1185  Maj-NL         San Francisco   54  225   
608   Christian Yelich   28   1185  Maj-NL             Milwaukee   58  247   
609       Andrew Young   26   1208  Maj-NL               Arizona   11   34   
610     Bradley Zimmer   27   1223  Maj-AL             Cleveland   20   50   
611        Mike Zunino   29   1186  Maj-AL             Tam

In [5]:
import pybaseball as pyb
import pandas as pd
import time  # Import the time module

all_data=[]
# Loop over the years from the beginning year to the current year
for year in range(2021, 2024):  # Replace 2024 with the current year
    try:
        data = pyb.batting_stats_bref(year)
        print(f"Data fetched for year: {year}")
        all_data.append(data)
    except Exception as e:
        print(f"An error occurred for year: {year}. Error: {str(e)}")
    
    time.sleep(4)  # Pause execution for 1 second

# Concatenate all dataframes in the list
all_data_df_4 = pd.concat(all_data)

print(all_data_df_4)

Data fetched for year: 2021
Data fetched for year: 2022
Data fetched for year: 2023
                  Name  Age  #days            Lev                   Tm    G  \
1          Cory Abbott   25    815         Maj-NL              Chicago    2   
2    Jos\xc3\xa9 Abreu   34    805         Maj-AL              Chicago  156   
3     Ronald Acuna Jr.   23    899         Maj-NL              Atlanta   82   
4         Willy Adames   25    805  Maj-AL,Maj-NL  Milwaukee,Tampa Bay  144   
5           Matt Adams   32    885         Maj-NL             Colorado   22   
..                 ...  ...    ...            ...                  ...  ...   
682   Masataka Yoshida   29     86         Maj-AL               Boston  140   
683        Jacob Young   23     86         Maj-NL           Washington   33   
684        Jared Young   27     86         Maj-NL              Chicago   16   
685        Seby Zavala   29     86  Maj-AL,Maj-NL      Arizona,Chicago   73   
686        Mike Zunino   32    195         Maj-

In [7]:
player_data=pd.DataFrame
player_data=pd.concat([all_data_df_1,all_data_df_2,all_data_df_3,all_data_df_4])


In [8]:
print(player_data)

                   Name  Age  #days            Lev               Tm    G   PA  \
1         David Aardsma   26   5659         Maj-AL           Boston    1    1   
2    Reggie Abercrombie   27   5567         Maj-NL          Houston   32   60   
3           Bobby Abreu   34   5567         Maj-AL         New York  156  684   
4          Manny Acosta   27   5651         Maj-NL          Atlanta    4    5   
5            Mike Adams   29   5596         Maj-NL        San Diego    2    2   
..                  ...  ...    ...            ...              ...  ...  ...   
682    Masataka Yoshida   29     86         Maj-AL           Boston  140  580   
683         Jacob Young   23     86         Maj-NL       Washington   33  121   
684         Jared Young   27     86         Maj-NL          Chicago   16   47   
685         Seby Zavala   29     86  Maj-AL,Maj-NL  Arizona,Chicago   73  193   
686         Mike Zunino   32    195         Maj-AL        Cleveland   42  140   

      AB    R    H  ...  SH